In [1]:
import sys
from pycparser import parse_file, c_parser, c_generator

filename = 'c-files/memmgr.c'

ast = parse_file(filename, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])





## Print original `memmgr.c` file

In [2]:
with open(filename, "r", encoding="utf8") as f:
    data = f.read()
    print(data)

//----------------------------------------------------------------
// Statically-allocated memory manager
//
// by Eli Bendersky (eliben@gmail.com)
//
// This code is in the public domain.
//----------------------------------------------------------------
#include "memmgr.h"

typedef ulong Align;

#define mem_header_union another_name

/*
    some comment
*/
union mem_header_union
{
    struct
    {
        // Pointer to the next block in the free list
        //
        union mem_header_union* next;

        // Size of the block (in quantas of sizeof(mem_header_t))
        //
        ulong size;
    } s;

    // Used to align headers in memory to a boundary
    //
    Align align_dummy;
};

typedef union mem_header_union mem_header_t;

// Initial empty list
//
static mem_header_t base;

// Start of free list
//
static mem_header_t* freep = 0;

// Static pool for new allocations
//
static byte pool[POOL_SIZE] = {0};
static ulong pool_free_pos = 0;


void memmgr_init()
{
    base.s.next

# Compile AST back to C

Note, that declarations from memmgr.h are incuded in the resulting code. 
However `#define` directives were lost, while `mem_header_union` was replaced with `another_name` due to 
```
#define mem_header_union another_name
``` 

See [c-files/memmgr.c](c-files/memmgr.c)

In [3]:
generator = c_generator.CGenerator()
print(generator.visit(ast))

typedef unsigned char byte;
typedef unsigned long ulong;
void memmgr_init();
void *memmgr_alloc(ulong nbytes);
void memmgr_free(void *ap);
void memmgr_print_stats();
typedef ulong Align;
union another_name
{
  struct 
  {
    union another_name *next;
    ulong size;
  } s;
  Align align_dummy;
};
typedef union another_name mem_header_t;
static mem_header_t base;
static mem_header_t *freep = 0;
static byte pool[8 * 1024] = {0};
static ulong pool_free_pos = 0;
void memmgr_init()
{
  base.s.next = 0;
  base.s.size = 0;
  freep = 0;
  pool_free_pos = 0;
}

static mem_header_t *get_mem_from_pool(ulong nquantas)
{
  ulong total_req_size;
  mem_header_t *h;
  if (nquantas < 16)
    nquantas = 16;

  total_req_size = nquantas * (sizeof(mem_header_t));
  if ((pool_free_pos + total_req_size) <= (8 * 1024))
  {
    h = (mem_header_t *) (pool + pool_free_pos);
    h->s.size = nquantas;
    memmgr_free((void *) (h + 1));
    pool_free_pos += total_req_size;
  }
  else
  {
    return 0;
  }

  retu